# Quick overview of the protocol
The protocol for anonymous transmission with bipartite links is as follows.


1. For all $i$ and $j$: Client $i$ prepares a secret key bit $k_{i}^{j}$ with client $j$ (skipping where $j = i$). Note that every pair $(i,j)$ shares two key bits: $k_{i}^{j}$ and $k_{j}^{i}$, which will be used for communication from $i$ to $j$, and from $j$ to $i$, respectively.
2. For all $i$: Client $i$ prepares $n$ random bits $\{r_{i}^{j}\}_{1 \leq j \leq n}$.
3. For all $i$: Client $i$ sets $r_{i}^{n} \rightarrow \bigoplus_{j = 1}^{n} r_{i}^{j}$, so that all bits together have even parity.
4. Alice sets $r_{a}^{n} \rightarrow r_{a}^{n} \oplus m$, so that the parity of all her bits is the bit message $m$.
5. For all $i$ and $j$: Client $i$ sends $r_{i}^{j} \oplus k_{i}^{j}$ to client $j$ through a public channel. They keep $r_{i}^{i}$ for themselves.
6. For all $i$ and $j$: Client $i$ receives $r_{j}^{i} \oplus k_{j}^{i}$ and decodes these towards $r_{j}^{i}$.
7. For all $i$: Client $i$ computes $z_{i} = \bigoplus_{j} r_{j}^{i}$ and announces this over the network.
8. For all $i$: Client $i$ computes $\bigoplus_{i} z_{i} = \bigoplus_{i}  \left(\bigoplus_{j} r_{j}^{i}\right) = \bigoplus_{j} \left(\bigoplus_{i} r_{i}^{j}\right) = 0 + 0 + \dots + m + 0 + \dots + 0 = m$

The quantum part of this protocol is to establish the secret key bits $k_{i}^{j}$. Every client needs a secret key bit with every other client (counting doubles!), so there are in total $n*(n-1)$ key bits needed. That means that there are an equal number of Bell pairs necessary (in the noiseless case).
The message that will be distributed will be longer, so that means that more key bits are necessary. Let the length of the message $m$ be $m$. In a real world scenario, the clients will establish enough bits between each other by repeating QKD enough. For a message of length $m$, every pair of clients needs to share $2m$ bits.

To establish the necessary key bits, every pair of clients will run QKD. The key rate of each client will determine the final announcement bit rate.

We make the assumption that the entire network is symmetrical: the clients have the same noise profiles, and their links to the server are all equally 'bad'.

Hence, we can calculate the key rate for one client pair separately, and extrapolate the total rate from this. Let this key rate be $k_{bi-part}$.
In principle, there are $\frac{n*(n-1)}{2}$ pairs of clients, and every pair needs 2 bits per message bit, so the final bitrate would be the single-pair bitrate, divided by $n*(n-1)$. However, this is assuming that all pairs of clients run their QKD schemes successively. Instead, it might be possible for some of these to run in parallel. If the server can server as many pairs together concurrently as possible (a 'simultaneous server') there's no squared order scaling, but less. Every pair still needs $2$ bits of key. If $n$ is even, every client can be server at the same time; there are then exactly $n - 1$ pairings necessary, times $2$ bits per pair. If $n$ is odd, one client is always left out; there are now $n$ pairings necessary (essentially the same for $n+1$). Hence, the scaling factor is $2 \times 2*\lceil\frac{n}{2}\rceil$. 

### Single client pair QKD
To perform QKD, the clients make use of bi-partite entanglement. More specifically, they ask the server to distribute $L$ Bell pairs $\ket{00} + \ket{11}$, where $L$ is referred to as the $\text{\emph{block size}}$. Using $L\times h_{2}(p)$ bits of pre-shared key, they randomly select $p\times L$ of the $L$ states to perform verification. For these rounds, they measure their qubits in the $X$ basis, and announce their measurement results. They compute the $X$-basis QBER from this as $Q_{X} = \frac{1 -\langle X_{A}X_{B}\rangle}{2}$. For the key rate calculations, we can calculate the $Z$-basis error rate from the round that would be used for key generation as $Q_{Z} = \frac{1 -\langle Z_{A}Z_{B}\rangle}{2}$.

## Trusted server
This is the most straightforward implementation. There is no need for $X$-basis (verification) steps, as the server is trusted and the clients assume they get a proper EPR pair. Therefore, they only have to perform $Z$-basis measurements, for both key generation, and a small portion for parameter estimation. More specifically than before, let there be $L$ rounds in total, of which $m$ are being used for parameter verification. These $m$ rounds are used to estimate the $Z$-basis QBER give an error rate $Q_{Z}$, that subsequently is taken as the $Z$-basis error rate of the rounds used for keygen as well - based on this, error correction is performed, which uses $h_{2}\left(Q_{Z}\right)\cdot L$ bits, that are publicly anounced (and therefore have to be 'subtracted' during error correction, as this public announcement becomes classical side information). 

Because the estimation rounds and keygen rounds have the same basis for their measurement, the clients can just measure everything in the same basis, and afterwards publicly decide on the subset of $m$ rounds.

There is a finite probability that the true error rate of the actual keygen rounds exceeds the estimate $Q_{Z}$. For error correction this is not a big issue: it will just mean that the error correction will fail, and that no key is being generated. However, a similar situation occurs in the untrusted setting for the estimate of the $Q_{X}$ (verification) error rate, from which the security of the key is derived. Hence, this issue needs to be addressed.

The method to circumvent this, is by including a so-called _statistical correction_ on $Q_{Z}$. This is a positive number $\mu$ that is added to $Q_{Z}$; one essentially overshoots the estimate by this amount, so that one can be increasingly sure that the true error rate lies below this overshot estimate. More quantitatively, the statistical correction $\mu$ is a function of $k, L-k$ and some security paramter $\varepsilon_{PE}$. For an estimated error rate $Q_{Z}$ during the estimation rounds, it then holds with probability $1 - \varepsilon_{PE}$ that the error rate of the keygen rounds does not exceed $Q_{Z} + \mu$. Such a statistical correction can therefore be used to obtain a quantative bound on the error correction failing (and, more importantly, for the $Q_{X}$ estimation in the untrusted setting).

Apart from the statistical correction, we do not take into account small finite-key corrections, as they are only a few bits (and don't grow in the block size). Out of $L$ EPR pairs, the number of secret key bits the client pair can create is then:
$$
(L - m) \cdot (1 - h_{2}\left(Q_{Z} + \mu_{Z}\right)).
$$

The number of announced anonymous bits is then this number, divided by $n \cdot (n - 1)$ if no client pairs can be 'catered to' simultaneously by the server, and divided by $2 \times 2*\lceil\frac{n}{2}\rceil$ in the simultaneous setting.

The function that computes this for a given set of parameters, is in `programs.EPR_based_trusted_server.functions`. It allows you to choose whether to perform a statistical correction on the PE (parameter estimation for the error correction) using `perform_statcor_PE`. Because it doesn't affect security (only the success rate), this is by default `False`. The `PE_tolerance` is the (exceedingly small) probability that the overshot of the error rate is not enough.

A run of this function is like so:
```
from programs.EPR_based_trusted_server.functions import get_number_announced_bits as EPR_trusted_length
EPR_trusted_lengths, durations = EPR_trusted_length(
                                    nr_clients=nr_clients,
                                    nr_rounds = nr_rounds_list[i],
                                    nr_estimation_rounds = nr_PE_rounds_list[i],
                                    perform_statcor_PE = do_PE_statist,
                                    network_configuration = network_config,
                                    nr_runtimes = nr_runtimes,
                                    )
```
This function takes as input parameters:
```
nr_clients:             (default 3) Number of clients. Used for extrapolation.
nr_rounds:              (default 1e3) Number of rounds to run.
nr_estimation_rounds:   (default 3e2) Number of rounds to consume to estimate the error rate.
perform_statcor_PE:     (default False) Whether to perform statistical correction to PE ereror rate.
PE_tolerance:           (default 1e-8) Tolerance for statistical correction of PE.
network_configuration:  (default None) Network configuration object from Squidasm.
nr_runtimes:            (default 1) Number of times to run the simulation.
Alice:                  (default None) Client that is Alice. Optional.
print_loop_nrs:         (default False) Print the loop number. Get's passed to the server program. Optional.
```

It outputs two nested lists: `message_lengths` and `run_times`. Each of these lists will have `nr_runs` items. For the `message_lengths`, every item is a list of two things: the message length in the simultaneous setting, and the message length in the  subsequent setting. The `run_times` is the same, but then for the total run times (i.e. the runtime of a single pair extrapolated). Both the message lengths and the runtimes are extrapolated from the single pair calculation, which the function actually simulates.

## Untrusted server

Now, the clients do not trust the server to actually share proper (although noisy) EPR pairs with them. As such, they need to perform privacy amplification on top of the error correction, which impacts the length of the announcements that they can do. Once again, we can start from a single pair, and extrapolate from there. To test their states (or rather, obtain an estimate on a non-commuting basis error rate), the clients now secretly arrange to utilise $k$ of the $L$ distributed states as verification rounds, by measuring in the $X$-basis instead of the $Z$-basis. To do this arrangement secretly, they use $h_{2}\left(\frac{k}{L}\right) \cdot L$ bits of pre-shared key. This needs to be subtracted from the final 'key'.

They compute the $X$-basis QBER from the verification rounds as $Q_{X}$. It is now important to perform a statistical correction on this rate, as not overshooting enough will affect security - however, there is still a parameter to not do the correction here as well.

If there is only white noise assumed, it is strictly speaking not necessary to perform separate parameter estimation - $Q_{X}$ and $Q_{Z}$ will be the same. However, these will be different in general (especially in adverserial models) so the function still assumes you want to do both. In the multipartite setting the bases are the same, so there it is an option to use the verification results for PE as well.

The number of usable bits for a single pair then becomes:

$$
(L - m - k) \cdot (1 - h_{2}\left(Q_{Z} + \mu_{Z}\right) - h_{2}\left(Q_{X} + \mu_{X}\right)) - L \cdot h_{2}\left(\frac{k}{L}\right)
$$
where the last term is the subtraction of the pre-shared key.

Once again this is then extrapolated based on the number of clients.

The function to calculate the total number of announced bits can then be found in `programs.EPR_based_untrusted_server.functions`. It allows you to do a statistical correction on both `PE` and `Verification` rounds (default `False` and `True`, resp.); you really should do it for `Verification`.

A run of this function is like so:
```
from programs.EPR_based_untrusted_server.functions import get_number_announced_bits as EPR_untrusted_length
EPR_untrusted_lengths, durations = EPR_untrusted_length(nr_clients = nr_clients,
                                    nr_rounds = nr_rounds_list[i],
                                    nr_verification_rounds = nr_VER_rounds_list[i],
                                    perform_statcor_VER = do_VER_statist,   
                                    VER_tolerance = VER_tolerance,
                                    nr_estimation_rounds = nr_PE_rounds_list[i],
                                    perform_statcor_PE = do_PE_statist,
                                    PE_tolerance = PE_tolerance,
                                    network_configuration = network_config,
                                    nr_runtimes = nr_runtimes,
    )
```
The function takes as input parameters:
```
nr_clients:             (default 3) Number of clients. Used for extrapolation.
nr_rounds:              (default 1e3) Number of rounds to run.
nr_verification_rounds: (default 3e2) Number of rounds to consume to perform verification.
perform_statcor_VER:    (default True) Whether to perform statistical correction to verification error rate.
VER_tolerance:          (default 1e-8) Tolerance for statistical correction of VER.
nr_estimation_rounds:   (default 3e2) Number of rounds to consume to estimate the error rate.
perform_statcor_PE:     (default False) Whether to perform statistical correction to PE ereror rate.
PE_tolerance:           (default 1e-8) Tolerance for statistical correction of PE.
network_configuration:  (default None) Network configuration object from Squidasm.
nr_runtimes:            (default 1) Number of times to run the simulation.
Alice:                  (default None) Client that is Alice. Optional.
print_loop_nrs:         (default False) Print the loop number. Get's passed to the server program. Optional.
```
It outputs the same as the trusted setting function.
